In [18]:
import subprocess
import os
import pandas as pd
import numpy as np
import tables as tb

# Mapowanie typów wiadomości PFCP na czytelne nazwy
pfcp_msg_type_map = {
    1: "heartbeat_request",
    2: "heartbeat_response",
    3: "pfd_management_request",
    4: "pfd_management_response",
    5: "association_setup_request",
    6: "association_setup_response",
    7: "association_update_request",
    8: "association_update_response",
    9: "association_release_request",
    10: "association_release_response",
    11: "version_not_supported_response",
    12: "node_report_request",
    13: "node_report_response",
    14: "session_set_deletion_request",
    15: "session_set_deletion_response",
    50: "session_establishment_request",
    51: "session_establishment_response",
    52: "session_modification_request",
    53: "session_modification_response",
    54: "session_deletion_request",
    55: "session_deletion_response",
    56: "session_report_request",
    57: "session_report_response",
}

class PcapCsvConverter:
    def __init__(self, data_directory, pcap_file, output_directory, interval=120):
        self.data_directory = data_directory
        self.pcap_file = os.path.join(data_directory, pcap_file)
        self.output_directory = output_directory
        self.interval = interval
        self.temp_csv_file = os.path.join(output_directory, 'temp_pcap.csv')

        # Wyodrębnij nazwę pliku bez rozszerzenia i dodaj rozszerzenie .csv z interwałem
        file_name = os.path.basename(pcap_file)
        file_base_name = os.path.splitext(file_name)[0]
        self.final_csv_file = os.path.join(output_directory, f'{file_base_name}_{interval}.csv')
        self.final_h5_file = os.path.join(output_directory, f'{file_base_name}_{interval}.h5')

    def convert_pcap_to_csv(self):
        command_csv = [
            'tshark',
            '-r', self.pcap_file,  # Plik wejściowy .pcap
            '-Y', 'pfcp',  # Filtr przechwytywania dla PFCP
            '-T', 'fields',
            '-E', 'separator=,',
            '-E', 'quote=d',
            '-e', 'frame.time_relative',  # Czas
            '-e', 'ip.src',  # Adres źródłowy
            '-e', 'ip.dst',  # Adres docelowy
            '-e', 'pfcp.msg_type',  # Typ wiadomości PFCP (jako liczba)
            '-e', 'frame.len',  # Długość ramki
            '-E', 'header=y'  # Nagłówek CSV
        ]

        with open(self.temp_csv_file, 'w') as file:
            subprocess.run(command_csv, stdout=file)  # Zapis do pliku CSV
        print(f"Conversion completed: {self.temp_csv_file}")

    def add_message_type_names(self):
        with open(self.temp_csv_file, 'r') as file:
            lines = file.readlines()

        with open(self.final_csv_file, 'w') as file:
            # Dodaj nagłówki kolumn
            file.write('frame.time,ip.src,ip.dst,pfcp.msg_type,frame.len,info\n')
            count = 0
            for line in lines[1:]:  # Pomijaj nagłówek
                parts = line.strip().split(',')
                text = parts[3].replace('"', '')
                try:
                    msg_type_name = pfcp_msg_type_map.get(int(text), 'unknown')  # Dodanie domyślnej wartości, jeśli klucz nie istnieje
                    # Dodaj nową kolumnę 'info' z nazwą typu wiadomości
                    file.write(f'{",".join(parts)},{msg_type_name}\n')
                except:
                    pass

    def process_csv(self):
        df = pd.read_csv(self.final_csv_file)

        min_time = df['frame.time'].min()
        max_time = df['frame.time'].max()

        # Utwórz etykiety przedziałów czasowych
        bins = np.arange(min_time, max_time + self.interval, self.interval)
        labels = [f"part_{i + 1}" for i in range(len(bins) - 1)]

        # Przydziel każdemu wierszowi odpowiedni przedział czasowy
        df['part'] = pd.cut(df['frame.time'], bins=bins, labels=labels, include_lowest=True)

        # Grupowanie i liczenie wartości 'info'
        df_grouped = df.groupby(['part', 'info']).size().reset_index(name='count')

        # Tworzenie tabeli przestawnej z wszystkimi wartościami ze słownika jako kolumny
        df_final = df_grouped.pivot_table(index='part', columns='info', values='count', fill_value=0)
        for msg_type_name in pfcp_msg_type_map.values():
            if msg_type_name not in df_final.columns:
                df_final[msg_type_name] = 0

        # Resetowanie indeksu i usuwanie kolumny 'part'
        df_final.reset_index(inplace=True)
        df_final.drop(columns=['part'], inplace=True)

        #zapis do csv
        df_final.to_csv(self.final_csv_file, index=False)

        '''
        # Zapis do pliku HDF5
        with tb.open_file(self.final_h5_file, mode='w') as h5file:
            h5file.create_table('/', 'pfcp_data', obj=df_final.to_records(index=False))
        print(f"Processed HDF5 saved as {self.final_h5_file}")

        '''
        print(f"Processed CSV saved as {self.final_csv_file}")
        

    def run(self):
        # Upewnienie się, że folder wyjściowy istnieje
        if not os.path.exists(self.output_directory):
            os.makedirs(self.output_directory)

        # Konwersja PCAP na CSV
        self.convert_pcap_to_csv()

        # Dodanie nazw typów wiadomości
        self.add_message_type_names()

        # Przetwarzanie i podział danych na przedziały czasowe
        self.process_csv()

        # Usunięcie tymczasowego pliku CSV
        os.remove(self.temp_csv_file)

def process_all_pcaps_in_directory(data_directory, output_directory, interval):
    # Upewnienie się, że folder wyjściowy istnieje
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Iteracja przez wszystkie pliki w podanym folderze
    processed_files_count = 0
    for filename in os.listdir(data_directory):
        if filename.endswith('.pcap'):
            print(f"Processing file: {filename}")
            processor = PcapCsvConverter(data_directory, filename, output_directory, interval)
            processor.run()
            processed_files_count += 1

    print(f"Total .pcap files processed: {processed_files_count}")

# Definicja parametrów bezpośrednio w notebooku
data_directory = 'test/'
output_directory = 'test/'
interval = 120

# Przetwarzanie wszystkich plików .pcap w podanym folderze i zapisywanie wyników w folderze docelowym
process_all_pcaps_in_directory(data_directory, output_directory, interval)


Processing file: captured_packets_1.pcap
Conversion completed: test/temp_pcap.csv
Processed CSV saved as test/captured_packets_1_120.csv
Processed HDF5 saved as test/captured_packets_1_120.h5
Processing file: test.pcap
Conversion completed: test/temp_pcap.csv
Processed CSV saved as test/test_120.csv
Processed HDF5 saved as test/test_120.h5
Total .pcap files processed: 2
